In [2]:
import matplotlib.pyplot as plt
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense, Dropout
from keras import optimizers
from keras.callbacks import ModelCheckpoint, EarlyStopping

In [3]:
# Initialing the CNN
classifier = Sequential()

# Step 1 - Convolution Layer
classifier.add(Convolution2D(
    32, (3,  3), input_shape=(64, 64, 3), activation='relu'))

# step 2 - Pooling
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Adding second convolution layer
classifier.add(Convolution2D(32, (3,  3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Adding 3rd Convolution Layer
classifier.add(Convolution2D(64, (3,  3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))


# Step 3 - Flattening
classifier.add(Flatten())

# Step 4 - Full Connection
classifier.add(Dense(256, activation='relu'))
classifier.add(Dropout(0.5))
classifier.add(Dense(30, activation='softmax'))

# Compiling The CNN
classifier.compile(
    optimizer=optimizers.SGD(learning_rate=0.01),
    loss='categorical_crossentropy',
    metrics=['accuracy'])

display(classifier.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPooling  (None, 6, 6, 64)         0

None

In [4]:
# Part 2 Fittting the CNN to the image
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory(
    '/kaggle/input/asl-dataset/training_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

test_set = test_datagen.flow_from_directory(
    '/kaggle/input/asl-dataset/test_set',
    target_size=(64, 64),
    batch_size=32,
    class_mode='categorical')

Found 47469 images belonging to 30 classes.
Found 6713 images belonging to 30 classes.


In [5]:
checkpoint = ModelCheckpoint('model_best_weight.h5', save_best_only=True, monitor='val_accuracy', mode='max')

In [6]:
model = classifier.fit(
    training_set,
    steps_per_epoch=800,
    epochs=25,
    validation_data=test_set,
    validation_steps=6500,
    callbacks=[checkpoint]
)

Epoch 1/25
800/800 [==============================] - 129s 150ms/step - loss: 2.2438 - accuracy: 0.3419 - val_loss: 0.6529 - val_accuracy: 0.8273
Epoch 2/25
800/800 [==============================] - 79s 98ms/step - loss: 0.6979 - accuracy: 0.7679
Epoch 3/25
800/800 [==============================] - 67s 83ms/step - loss: 0.4230 - accuracy: 0.8568
Epoch 4/25
800/800 [==============================] - 61s 77ms/step - loss: 0.3229 - accuracy: 0.8887
Epoch 5/25
800/800 [==============================] - 59s 73ms/step - loss: 0.2543 - accuracy: 0.9126
Epoch 6/25
800/800 [==============================] - 58s 72ms/step - loss: 0.2218 - accuracy: 0.9232
Epoch 7/25
800/800 [==============================] - 58s 72ms/step - loss: 0.1930 - accuracy: 0.9332
Epoch 8/25
800/800 [==============================] - 57s 72ms/step - loss: 0.1690 - accuracy: 0.9421
Epoch 9/25
800/800 [==============================] - 56s 70ms/step - loss: 0.1483 - accuracy: 0.9479
Epoch 10/25
800/800 [=================

In [7]:
scores = classifier.evaluate(test_set)
print("%s: %.2f%%" % ("Evaluate Test Accuracy", scores[1]*100))

210/210 [==============================] - 13s 61ms/step - loss: 0.0754 - accuracy: 0.9891
Evaluate Test Accuracy: 98.91%


In [8]:
# Visualization
import matplotlib
import matplotlib.pyplot as plt
import plotly
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
from sklearn.manifold import TSNE
import numpy as np

In [9]:
history = model
# Visualize Training and Validation Results

# Create Subplot
fig = make_subplots(
        rows=1, cols=2,
        subplot_titles=["Model Loss", "Model Accuracy"], 
)

# Configuration Plot
class PlotCFG:
    marker_size = 8
    line_size = 2
    train_color = "#76503d"
    valid_color = "#deb392"

# Loss Plot
loss = history.history['loss']
val_loss = history.history['val_loss']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(loss)+1), y=loss,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Loss"
    ), row=1, col=1
)


# Accuracy Plot
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
fig.add_trace(
    go.Scatter(
        x=np.arange(1, len(acc)+1), y=acc,
        mode="markers+lines",
        marker=dict(
            color=PlotCFG.train_color, size=PlotCFG.marker_size,
            line=dict(color="White", width=0.5)
        ),
        line=dict(color=PlotCFG.train_color, width=PlotCFG.line_size),
        name="Training Accuracy"
    ), row=1, col=2
)


# Update Axes
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_xaxes(title="Epochs", linecolor="Black", ticks="outside", row=1, col=2)
fig.update_yaxes(title="Categorical Loss", linecolor="Black", ticks="outside", row=1, col=1)
fig.update_yaxes(title="Accuracy", linecolor="Black", ticks="outside", row=1, col=2)

# Update Layout
fig.update_layout(
    title="Training Loss and Metrics", title_x=0.5,
    width=950, height=400,
    showlegend=False,
    plot_bgcolor="White",
    paper_bgcolor="White"
)

# Show
fig.show(iframe_connected=True)